In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.drop(columns=['Outcome'])
y = df['Outcome']

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [11]:
model = Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['Accuracy'])

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 20ms/step - loss: 0.6529 - Accuracy: 0.6466 - val_loss: 0.6276 - val_Accuracy: 0.6948
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6034 - Accuracy: 0.6808 - val_loss: 0.5860 - val_Accuracy: 0.7273
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5685 - Accuracy: 0.7231 - val_loss: 0.5533 - val_Accuracy: 0.7532
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5440 - Accuracy: 0.7329 - val_loss: 0.5273 - val_Accuracy: 0.7792
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5239 - Accuracy: 0.7362 - val_loss: 0.5115 - val_Accuracy: 0.7922
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5094 - Accuracy: 0.7394 - val_loss: 0.4995 - val_Accuracy: 0.8117
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4992 - Accuracy: 0.7410 - val_loss: 0.4881 - val_Accuracy: 0.8052
Epoch 8/100


In [13]:
import keras_tuner as kt

# For optimizer

In [14]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [15]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='tuner-files-for-optimizer')

In [16]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 05s
INFO:tensorflow:Oracle triggered exit


In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
model = tuner.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.fit(X_train,y_train,initial_epoch=6,epochs=100,batch_size=32,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 9ms/step - loss: 0.5326 - accuracy: 0.7476 - val_loss: 0.5080 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5188 - accuracy: 0.7573 - val_loss: 0.4978 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5081 - accuracy: 0.7508 - val_loss: 0.4893 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4995 - accuracy: 0.7590 - val_loss: 0.4831 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4926 - accuracy: 0.7622 - val_loss: 0.4788 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4863 - accuracy: 0.7671 - val_loss: 0.4738 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4809 - accuracy: 0.7687 - val_loss: 0.4710 - val_accuracy: 0.7987
Epoch 14/

# For number of neurons

In [21]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',min_value=8,max_value=128,step=8)    # (start,end,step)
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [22]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='tuner-files-for-units')

In [23]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 06s
INFO:tensorflow:Oracle triggered exit


In [24]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [25]:
model = tuner.get_best_models(num_models=1)[0]

In [28]:
model.fit(X_train, y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4070 - accuracy: 0.8160
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4064 - accuracy: 0.8192
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4066 - accuracy: 0.8143
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4058 - accuracy: 0.8192
Epoch 11/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4053 - accuracy: 0.8127
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4053 - accuracy: 0.8176
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4044 - accuracy: 0.8127
Epoch 14/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4042 - accuracy: 0.8160
Epoch 15/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4034 - accuracy: 0.8192
Epoch 16/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4035 - accuracy: 0.8176
Epo

# For layers

In [29]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [30]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='tuner-files-for-layers')

In [31]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 08s
INFO:tensorflow:Oracle triggered exit


In [33]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [34]:
model = tuner.get_best_models(num_models=1)[0]

In [35]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4710 - accuracy: 0.7704 - val_loss: 0.4557 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4552 - accuracy: 0.7932 - val_loss: 0.4818 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4450 - accuracy: 0.7997 - val_loss: 0.4574 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4312 - accuracy: 0.8013 - val_loss: 0.4493 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4221 - accuracy: 0.8046 - val_loss: 0.4535 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4127 - accuracy: 0.8290 - val_loss: 0.4946 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3988 - accuracy: 0.8094 - val_loss: 0.4778 - val_accuracy: 0.8117
Epoch 14

# Full model tuning

In [49]:
def build_model(hp):
    model = Sequential()
    counter = 0

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(
                Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                      input_dim=8)
            )
            counter+=1
        else:
            model.add(
                Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']))
            )
    model.add(Dense(1,activation='sigmoid'))
    model.compile(hp.Choice('optimizer',values=['rmsprop','adam','nadam','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [50]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='tuner-files-for-full-model')

In [51]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 01s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 07s
INFO:tensorflow:Oracle triggered exit


In [52]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 64,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 128,
 'activation1': 'tanh',
 'units2': 16,
 'activation2': 'tanh',
 'units3': 8,
 'activation3': 'relu',
 'units4': 96,
 'activation4': 'tanh',
 'units5': 112,
 'activation5': 'relu',
 'units6': 104,
 'activation6': 'sigmoid',
 'units7': 24,
 'activation7': 'relu',
 'units8': 128,
 'activation8': 'sigmoid'}

In [53]:
model=tuner.get_best_models(num_models=1)[0]

In [54]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 [==============================] - 1s 10ms/step - loss: 0.5093 - accuracy: 0.7459 - val_loss: 0.4875 - val_accuracy: 0.7987
Epoch 8/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4928 - accuracy: 0.7606 - val_loss: 0.4816 - val_accuracy: 0.7987
Epoch 9/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4839 - accuracy: 0.7736 - val_loss: 0.4759 - val_accuracy: 0.7857
Epoch 10/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4771 - accuracy: 0.7801 - val_loss: 0.4712 - val_accuracy: 0.7857
Epoch 11/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4741 - accuracy: 0.7720 - val_loss: 0.4739 - val_accuracy: 0.7922
Epoch 12/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4704 - accuracy: 0.7736 - val_loss: 0.4713 - val_accuracy: 0.7792
Epoch 13/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4687 - accuracy: 0.7720 - val_loss: 0.4695 - val_accuracy: 0.7857
Epoch 14